In [1]:
import numpy as np
import pandas as pd
import time 
import xgboost as xgb
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder

d:\code environment\python\python3.6\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['font.serif'] = ['SimHei']
import seaborn as sns
%matplotlib inline
p = sns.color_palette()
sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})

In [3]:
# sns.FacetGrid(dataset, hue="标签", size=5).map(plt.scatter, 1, "标签").add_legend()
# xxx=dataset[['标签',1,27,31,38,40]]
# sns.pairplot(xxx, hue="标签", size=2)

** 读入数据：**

In [4]:
#a1 = OneHotEncoder(sparse = False).fit_transform( trains[['用户性别']] )
#a2 = OneHotEncoder(sparse = False).fit_transform( trains[['用户职业']])
#a3 = OneHotEncoder(sparse = False).fit_transform( trains[['用户教育程度']])
#a4 = OneHotEncoder(sparse = False).fit_transform( trains[['用户婚姻状态']])
#a5 = OneHotEncoder(sparse = False).fit_transform( trains[['用户户口类型']])
#final_output = np.hstack((a1,a2,a3,a4,a5))
#final_output

#pandas 自带的get_dummies函数实现one-hot,并构造组合特征
# trains = pd.read_csv("../feature/训练表_20170119_A.csv",encoding="gb2312") # 注意自己数据路径
# #trains.index = trains['用户标识'].tolist()
# train_label=trains[['用户标识','标签']]
# trains=pd.get_dummies(trains,columns=trains[['用户性别','用户职业','用户教育程度','用户婚姻状态','用户户口类型']]).drop(['标签'],axis=1)

# tests = pd.read_csv("../feature/测试表_20170119_A.csv",encoding="gb2312") # 注意自己数据路径
# #tests.index = tests['用户标识'].tolist()
# test_label=tests[['用户标识','标签']]
# tests=pd.get_dummies(tests,columns=tests[['用户性别','用户职业','用户教育程度','用户婚姻状态','用户户口类型']]).drop(['标签'],axis=1)

# from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures(interaction_only=True)
# trains=poly.fit_transform(trains)
# df = pd.DataFrame(trains.reshape(55596, 326))
# df.interpolate(axis=1).values.reshape(trains.shape)
# df['用户标识']=df[1].astype(int)
# trains=pd.merge(train_label,df,how='inner', on = "用户标识")

# tests=poly.fit_transform(tests)
# df = pd.DataFrame(tests.reshape(13899, 326))
# df.interpolate(axis=1).values.reshape(tests.shape)
# df['用户标识']=df[1].astype(int)
# tests=pd.merge(test_label,df,how='inner', on = "用户标识")

## 用户信息表

In [5]:
train = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/test.csv",encoding="gb2312") 
test = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/train.csv",encoding="gb2312") 
test=test.fillna(-1)
dataset= pd.concat([train,test],axis=0).reset_index()
label=dataset[['用户标识','标签']]
dataset=dataset.drop(['index','用户标识'],axis=1)

dataset=pd.get_dummies(dataset,columns=dataset[['用户性别','用户职业','用户教育程度','用户婚姻状态','用户户口类型']]).drop(['标签'],axis=1)
#dataset 
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3,interaction_only=True) #之前設置的是2，现在改成3，更多的交互特征
dataset=poly.fit_transform(dataset)
df = pd.DataFrame(dataset.reshape(23899,2325))
df.interpolate(axis=1).values.reshape(dataset.shape)

#dataset
#trains=pd.merge(label,df,left_index=True,right_index=True,how='outer')
trains=pd.concat([label,df],axis=1)
trains.to_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/feature_total_A.csv",index=None,encoding="gb2312") #mean填充 69495 rows × 1171 columns 没有删去用户标识

## 用户浏览行为训练表

In [6]:
train = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/user_browse_test.csv",encoding="gb2312") 
test = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/user_browse_train.csv",encoding="gb2312") 
dataset= pd.concat([train,test],axis=0)
temp=dataset[:]
#统计行和列的缺失值数
dataset=dataset.fillna(-1)
temp['用户浏览行为缺失统计']=(dataset==-1).sum(axis=1)
#用均值填充缺失值
temp.fillna(temp.mean(),inplace=True)
trains=pd.merge(trains,temp,how='left', on = "用户标识")

## 用户银行流水记录表

In [7]:
train = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/bank_train.csv",encoding="gb2312") # 注意自己数据路径
test = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/bank_test.csv",encoding="gb2312") # 注意自己数据路径
dataset= pd.concat([train,test],axis=0)
temp=dataset[:]
#统计行和列的缺失值数
dataset=dataset.fillna(-1)
temp['用户银行流水记录缺失统计']=(dataset==-1).sum(axis=1)
#用均值填充缺失值
temp.fillna(temp.mean(),inplace=True)
temp
trains=pd.merge(trains,temp,how='left', on = "用户标识")

## 用户账单表初始特征

In [8]:
train = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/bill_prifeature_train.csv",encoding="gb2312") # 注意自己数据路径
test = pd.read_csv("D:/Files/QQFiles/935257580/FileRecv/personal loan/feature/bill_prifeature_test.csv",encoding="gb2312") # 注意自己数据路径
dataset= pd.concat([train,test],axis=0)
temp=dataset[:]
print(temp.shape)
#统计行和列的缺失值数
dataset=dataset.fillna(-1)
temp['用户银行流水记录缺失统计']=(dataset==-1).sum(axis=1)
#特征轮：构造放款后与放款前差值特征
feature_name=temp.columns 
for name in feature_name:
    if name.find('款后')> 0:
        x1=name.replace('款后','款前')
        if x1 in feature_name:
            temp[name+"与"+x1+"差值"]=temp[name]-temp[x1]        
#用均值填充缺失值
temp.fillna(temp.mean(),inplace=True)
#temp

trains=pd.merge(trains,temp,how='left', on = "用户标识")

(23899, 57)


## 用户账单表特征

In [9]:
train = pd.read_csv("../feature/用户账单表特征训练表20170119_A.csv",encoding="gb2312") # 注意自己数据路径
test = pd.read_csv("../feature/用户账单表特征测试表20170119_A.csv",encoding="gb2312") # 注意自己数据路径
dataset= pd.concat([train,test],axis=0)
temp=dataset[:]
print(temp.shape)#(69495, 466)
#统计行和列的缺失值数
dataset=dataset.fillna(-1)
temp['用户银行流水记录缺失统计']=(dataset==-1).sum(axis=1)
#特征轮：构造放款后与放款前差值特征
feature_name=temp.columns 
for name in feature_name:
    if name.find('款后')> 0:
        x1=name.replace('款后','款前')
        if x1 in feature_name:
            temp[name+"与"+x1+"差值"]=temp[name]-temp[x1]        
#用均值填充缺失值
temp.fillna(temp.mean(),inplace=True)
temp#69495 rows × 699 columns

trains=pd.merge(trains,temp,how='left', on = "用户标识")

FileNotFoundError: File b'../feature/\xe7\x94\xa8\xe6\x88\xb7\xe8\xb4\xa6\xe5\x8d\x95\xe8\xa1\xa8\xe7\x89\xb9\xe5\xbe\x81\xe8\xae\xad\xe7\xbb\x83\xe8\xa1\xa820170119_A.csv' does not exist

In [ ]:
#trains.to_csv("../feature/特征汇总_20170119_A.csv",index=None,encoding="gb2312") #mean填充 69495 rows × 1171 columns 没有删去用户标识
#trains.to_csv("../feature/特征汇总_20170119_B.csv",index=None,encoding="gb2312") #0填充  69495 rows × 1171 columns 没有删去用户标识

#trains.to_csv("../feature/特征汇总_20170119_D.csv",index=None,encoding="gb2312") #0填充 69495 rows × 1146 columns 当前最高分0.43267
#trains.to_csv("../feature/特征汇总_20170119_E.csv",index=None,encoding="gb2312") #mean填充 69495 rows × 1146 columns

### 特征提取

In [ ]:
sns.FacetGrid(dataset, hue="标签", size=5).map(plt.scatter, 1, "标签").add_legend()
xxx=dataset[['标签',1,27,31,38,40]]
sns.pairplot(xxx, hue="标签", size=2)